In [2]:
# Install dependencies (only needed in local environments)
!pip install gradio deepface opencv-python numpy

In [3]:
import gradio as gr
import cv2
import numpy as np
from deepface import DeepFace

# PHQ-9 Questions
phq9_questions = [
    "Little interest or pleasure in doing things?",
    "Feeling down, depressed, or hopeless?",
    "Trouble falling or staying asleep, or sleeping too much?",
    "Feeling tired or having little energy?",
    "Poor appetite or overeating?",
    "Feeling bad about yourself?",
    "Trouble concentrating on things?",
    "Moving or speaking so slowly?",
    "Thoughts of self-harm?"
]

# Answer options for PHQ-9
options = {
    "Not at all": 0,
    "Several days": 1,
    "More than half the days": 2,
    "Nearly every day": 3
}

# Mental health suggestions
suggestions = {
    "happy_Low": "You're doing great! Stay active and positive. Engaging in social activities, pursuing hobbies, and maintaining a balanced lifestyle can enhance your well-being. Keep up the positive energy!",
    "happy_Moderate": "Try to maintain a healthy balance between work and relaxation. Consider practicing mindfulness, setting achievable goals, and ensuring regular social interactions to support your happiness.",
    "happy_High": "Even happy people need support sometimes. Stay connected with loved ones, engage in stress-relief activities, and be mindful of emotional fluctuations. If needed, seeking professional guidance can enhance overall well-being.",
    "neutral_Low": "Stay engaged with hobbies and interests. Finding joy in small activities, maintaining a routine, and practicing self-care can promote a steady and positive mindset.",
    "neutral_Moderate": "Mindfulness and relaxation techniques may help improve your mood. Consider journaling, deep breathing exercises, or seeking engaging activities to enhance emotional resilience.",
    "neutral_High": "Consider seeking professional support for better well-being. Speaking with a therapist, joining support groups, or exploring structured wellness programs can be beneficial.",
    "sad_Low": "Stay connected with friends and engage in self-care activities. Taking small steps to address emotional well-being, such as listening to music or engaging in light exercise, can help improve mood.",
    "sad_Moderate": "Try small daily goals to feel accomplished. Breaking down tasks, rewarding achievements, and staying active can gradually uplift your mental state.",
    "sad_High": "If you're struggling, reach out to a trusted friend, family member, or professional. Mental health support and therapy can provide effective tools to navigate emotional challenges. Remember, you're not alone!"
}

def analyze_mental_health(image, emojis, *answers):
    """Analyze emotion from the image, detect manual emojis, and provide mental health suggestions."""
    captured_emotion = "neutral"  # Default emotion

    # Image-based emotion detection using DeepFace
    if image is not None:
        try:
            image = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)  # Convert to OpenCV format
            analysis = DeepFace.analyze(image, actions=['emotion'], enforce_detection=False)
            detected_emotion = analysis[0]['dominant_emotion'].lower()

            # Convert DeepFace emotions to our categories
            if detected_emotion in ["happy", "surprise"]:
                captured_emotion = "happy"
            elif detected_emotion in ["sad", "fear", "angry"]:
                captured_emotion = "sad"
            else:
                captured_emotion = "neutral"
        except:
            captured_emotion = "neutral"

    # Process manually entered emojis
    if any(emoji in emojis for emoji in ["😂", "😊", "😁", "😃"]):  # Positive emojis
        captured_emotion = "happy"
    elif any(emoji in emojis for emoji in ["😢", "😞", "😭", "😔"]):  # Negative emojis
        captured_emotion = "sad"

    # Calculate PHQ-9 score
    total_score = sum([options[answer] for answer in answers])
    severity = "Low" if total_score <= 4 else "Moderate" if total_score <= 14 else "High"

    final_key = f"{captured_emotion}_{severity}"
    suggestion = suggestions.get(final_key, "Stay positive and take care of yourself! ❤")

    return f"Detected Emotion: {captured_emotion.capitalize()} (via Emojis: {emojis})\nPHQ-9 Score: {total_score}/27\nSuggestion: {suggestion}"

# Gradio UI (File upload + Manual Emoji Input)
inputs = [gr.Image(type="pil", label="Upload Your Image"),
          gr.Textbox(label="Enter Emojis (e.g., 😂😊😢)")]

inputs.extend([gr.Radio(choices=list(options.keys()), label=q) for q in phq9_questions])

output = gr.Textbox(label="Mental Health Suggestion")

iface = gr.Interface(
    fn=analyze_mental_health,
    inputs=inputs,
    outputs=output,
    title="Mental Health Analysis",
    description="Upload an image, enter emojis, answer the PHQ-9 questions, and receive mental health insights."
)

iface.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://2f98cb75b14cf0d7cd.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
